In [1]:
import gmaps
import pandas as pd
import numpy as np
import json
import seaborn
import requests
import matplotlib.pyplot as plt

# Google developer API key
from api_key import gmap_api

# Access maps with unique API key
gmaps.configure(api_key=gmap_api)

In [2]:
#read in combined csvfile
all_agencies_df = pd.read_csv("../data/cleandata/all_year_agencies.csv")

In [3]:
clean_all_agencies_df = all_agencies_df.dropna(how="any")

#limit  data to year of natural disaster
louisa_agencies_pd = clean_all_agencies_df.loc[clean_all_agencies_df["report_year"] == 2010, :]
louisa_agencies_pd.head()

,Source.Name,report_year,agency_id,ori,agency_name,state_abbr,primary_county_id,primary_county,population,nibrs_months_reported,icpsr_zip,icpsr_lat,icpsr_lng,coordinates
6520,TX2010cde_agencies.csv,2010,19425,TX1350000,King County Sheriff's Office,TX,2749,King,257,0,79236.0,33.614159,-100.250548,"(33.614159, -100.250548)"
6521,TX2010cde_agencies.csv,2010,19561,TX1730000,Motley County Sheriff's Office,TX,2787,Motley,1142,0,79244.0,34.057899,-100.793152,"(34.057899, -100.793152)"
6522,TX2010cde_agencies.csv,2010,19209,TX0920500,Longview Police Department,TX,2706,Gregg,81842,12,75606.0,32.486397,-94.816276,"(32.486397, -94.816276)"
6523,TX2010cde_agencies.csv,2010,19639,TX1990300,Heath Police Department,TX,2813,Rockwall,8176,12,75032.0,32.898054,-96.404396,"(32.898054, -96.404396)"
6524,TX2010cde_agencies.csv,2010,19009,TX0570100,Addison Police Department,TX,2671,Dallas,16125,0,75001.0,32.766987,-96.778424,"(32.766987, -96.778424)"


In [4]:
#limit  data to state of natural disaster
va_agencies_pd = louisa_agencies_pd.loc[louisa_agencies_pd["state_abbr"] == "VA", :]
va_agencies_pd.head()

,Source.Name,report_year,agency_id,ori,agency_name,state_abbr,primary_county_id,primary_county,population,nibrs_months_reported,icpsr_zip,icpsr_lat,icpsr_lng,coordinates
15597,VA2010cde_agencies.csv,2010,20479,VA1300000,Williamsburg Police Department,VA,3032,Williamsburg City,14915,12,23185.0,37.269524,-76.708165,"(37.269524, -76.708165)"
15598,VA2010cde_agencies.csv,2010,20193,VA0470000,James City County Sheriff's Office,VA,2966,James City,0,0,23185.0,37.324427,-76.778319,"(37.324427, -76.778319)"
15599,VA2010cde_agencies.csv,2010,20225,VA056SP00,State Police: Madison County,VA,2977,Madison,0,12,22701.0,38.412059,-78.276961,"(38.412059, -78.276961)"
15601,VA2010cde_agencies.csv,2010,20466,VA1230000,Roanoke Police Department,VA,3010,Roanoke City,99827,12,24016.0,37.278458,-79.958174,"(37.278458, -79.958174)"
15602,VA2010cde_agencies.csv,2010,20129,VA0290100,Fairfax County Police Department,VA,2940,Fairfax,1106066,12,22030.0,38.833742,-77.276117,"(38.833742, -77.276117)"


In [5]:
# show only participating agencies
participant_va_agencies_df = va_agencies_pd.loc[va_agencies_pd["nibrs_months_reported"] == 12, :]
participant_va_agencies_df = pd.DataFrame(participant_va_agencies_df)
participant_va_agencies_df.head()

,Source.Name,report_year,agency_id,ori,agency_name,state_abbr,primary_county_id,primary_county,population,nibrs_months_reported,icpsr_zip,icpsr_lat,icpsr_lng,coordinates
15597,VA2010cde_agencies.csv,2010,20479,VA1300000,Williamsburg Police Department,VA,3032,Williamsburg City,14915,12,23185.0,37.269524,-76.708165,"(37.269524, -76.708165)"
15599,VA2010cde_agencies.csv,2010,20225,VA056SP00,State Police: Madison County,VA,2977,Madison,0,12,22701.0,38.412059,-78.276961,"(38.412059, -78.276961)"
15601,VA2010cde_agencies.csv,2010,20466,VA1230000,Roanoke Police Department,VA,3010,Roanoke City,99827,12,24016.0,37.278458,-79.958174,"(37.278458, -79.958174)"
15602,VA2010cde_agencies.csv,2010,20129,VA0290100,Fairfax County Police Department,VA,2940,Fairfax,1106066,12,22030.0,38.833742,-77.276117,"(38.833742, -77.276117)"
15603,VA2010cde_agencies.csv,2010,20070,VA0090000,Bath County Sheriff's Office,VA,2910,Bath,4513,12,24484.0,38.068988,-79.732898,"(38.068988, -79.732898)"


In [6]:
# Store 'Lat' and 'Lng' into  locations 
part_va_locations = participant_va_agencies_df[["icpsr_lat", "icpsr_lng"]].astype(float)
part_va_locations.count()

icpsr_lat    350
icpsr_lng    350
dtype: int64

In [7]:
# Store 'Lat' and 'Lng' into  locations 
va_locations = va_agencies_pd[["icpsr_lat", "icpsr_lng"]].astype(float)
va_locations.count()

icpsr_lat    380
icpsr_lng    380
dtype: int64

In [8]:
# Customize the size of the figure
part_figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
part_fig = gmaps.figure(layout=part_figure_layout)

In [9]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(va_locations,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

# Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer.dissipating = False
heat_layer.max_intensity = 100
heat_layer.point_radius = 2

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(part_va_locations)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))